In [21]:
import plotly.graph_objects as go
import backtrader as bt
import pandas as pd
import matplotlib.pyplot as plt
import yfinance as yf
from IPython.display import display


# **Data**

In [22]:
ticker = "BTC-USD"

data = yf.download(ticker, start="2020-01-01", end="2024-12-27", auto_adjust=True)

data.columns = [col[0] if col[1] == '' else col[0] for col in data.columns]


columns_to_keep = ['Open', 'High', 'Low', 'Close', 'Volume']
data = data[columns_to_keep]

data

[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Volume
Date,,,,,
2020-01-01,7194.892090,7254.330566,7174.944336,7200.174316,18565664997
2020-01-02,7202.551270,7212.155273,6935.270020,6985.470215,20802083465
2020-01-03,6984.428711,7413.715332,6914.996094,7344.884277,28111481032
2020-01-04,7345.375488,7427.385742,7309.514160,7410.656738,18444271275
2020-01-05,7410.451660,7544.497070,7400.535645,7411.317383,19725074095
...,...,...,...,...,...
2024-12-22,97218.320312,97360.265625,94202.187500,95104.937500,43147981314
2024-12-23,95099.390625,96416.210938,92403.132812,94686.242188,65239002919
2024-12-24,94684.343750,99404.062500,93448.015625,98676.093750,47114953674


# **Strategies**

In [23]:
class MyStrategy(bt.Strategy):

    params = (
        ('period', 20),
        ('zscore_upper', 1),   
        ('zscore_lower', -1),  
    )

    def __init__(self):
        self.buy_signals = []
        self.sell_signals = []

        self.sma = bt.indicators.SimpleMovingAverage(period=50)
        self.std = bt.indicators.StandardDeviation(self.data.close, period=50)
        self.zscore = (self.data.close - self.sma) / self.std

        self.upper_bound = self.sma + self.std
        self.lower_bound = self.sma - self.std

    def next(self):
        if not self.position and self.data.close < self.lower_bound and self.zscore < self.params.zscore_lower:
            self.buy()
            self.buy_signals.append((self.data.datetime.datetime(0), self.data.close[0]))

        elif self.position and self.data.close > self.upper_bound and self.zscore > self.params.zscore_upper:
            self.sell()
            self.sell_signals.append((self.data.datetime.datetime(0), self.data.close[0]))

class BuyAndHold(bt.Strategy):
    def __init__(self):
        self.buy_signals = []
        self.sell_signals = []

    def nextstart(self):
        self.order = self.buy()
        self.buy_signals.append((self.data.datetime.datetime(0), self.data.close[0]))

# **BackTrader**

In [24]:
strategies = [MyStrategy, BuyAndHold]

strategy_dict = {}

for strategy in strategies:
    
    cerebro = bt.Cerebro()

    cerebro.addstrategy(strategy)

    data_feed = bt.feeds.PandasData(dataname=data)

    cerebro.adddata(data_feed)

    cerebro.broker.setcash(100000)

    cerebro.broker.setcommission(commission=0.001)

    cerebro.addanalyzer(bt.analyzers.TimeReturn, _name="timereturn")
    cerebro.addanalyzer(bt.analyzers.Returns, _name="returns")
    cerebro.addanalyzer(bt.analyzers.DrawDown, _name="drawdown")

    results = cerebro.run()

    strategy_dict[strategy.__name__] = {
        'cerebro': cerebro,
        'results': results
    }

In [25]:
# rtot: The total return for the entire backtest period (29.66%)
# ravg: The average per-bar return (e.g., if these are daily bars, this is the average daily return)
# rnorm: The normalized (annualized) return (23.00% per year, if daily bars)
# rnorm100: The same normalized return as a percentage (23.00%)

strategy_dict['MyStrategy']['results'][0].analyzers.returns.get_analysis()

OrderedDict([('rtot', -0.11627539254766263),
             ('ravg', -6.381744925777312e-05),
             ('rnorm', -0.015953372333835338),
             ('rnorm100', -1.5953372333835338)])

In [26]:
# 1) The current drawdown length (in bars).
# 2) The current drawdown in percentage (i.e., ~6.11%).
# 3) The current drawdown in monetary terms (e.g., $8,753.39).
# 4) The length (in bars) of the maximum drawdown observed.
# 5) The maximum drawdown percentage (~20.83%).
# 6) The maximum drawdown in monetary terms (e.g., $27,025.29).

strategy_dict['MyStrategy']['results'][0].analyzers.drawdown.get_analysis()

AutoOrderedDict([('len', 1328),
                 ('drawdown', 17.333027883648683),
                 ('moneydown', 18665.715560546858),
                 ('max',
                  AutoOrderedDict([('len', 1328),
                                   ('drawdown', 42.284295440057164),
                                   ('moneydown', 45535.415777343755)]))])

In [27]:
strategy_dict['BuyAndHold']['results'][0].analyzers.drawdown.get_analysis()

AutoOrderedDict([('len', 9),
                 ('drawdown', 5.200342759777772),
                 ('moneydown', 10345.0859375),
                 ('max',
                  AutoOrderedDict([('len', 846),
                                   ('drawdown', 32.29015256731743),
                                   ('moneydown', 51779.5439453125)]))])

In [28]:
strategy_dict['BuyAndHold']['results'][0].analyzers.timereturn.get_analysis()

OrderedDict([(datetime.datetime(2020, 1, 1, 0, 0), 0.0),
             (datetime.datetime(2020, 1, 2, 0, 0), -0.002242836059570208),
             (datetime.datetime(2020, 1, 3, 0, 0), 0.00360221981349218),
             (datetime.datetime(2020, 1, 4, 0, 0), 0.0006568370225077391),
             (datetime.datetime(2020, 1, 5, 0, 0), 6.593199382676218e-06),
             (datetime.datetime(2020, 1, 6, 0, 0), 0.003571819068943549),
             (datetime.datetime(2020, 1, 7, 0, 0), 0.003922784820837633),
             (datetime.datetime(2020, 1, 8, 0, 0), -0.0008303746345890151),
             (datetime.datetime(2020, 1, 9, 0, 0), -0.001990594696013037),
             (datetime.datetime(2020, 1, 10, 0, 0), 0.0028557153002144453),
             (datetime.datetime(2020, 1, 11, 0, 0), -0.0012779386933708947),
             (datetime.datetime(2020, 1, 12, 0, 0), 0.0015368436869711566),
             (datetime.datetime(2020, 1, 13, 0, 0), -0.00047829762328222003),
             (datetime.datetime(2020, 1

In [29]:
df = data.copy()
df['Date'] = df.index

run_results = strategy_dict['MyStrategy']['results']
strategy_instance = run_results[0]

buys = strategy_instance.buy_signals
sells = strategy_instance.sell_signals

fig = go.Figure()

fig.add_trace(go.Scatter(
    x=df['Date'],
    y=df['Close'],
    mode='lines',
    name='Price'
))

fig.add_trace(go.Scatter(
    x=[signal[0] for signal in buys],
    y=[signal[1] for signal in buys],
    mode='markers',
    marker=dict(size=10, color='green', symbol='triangle-up'),
    name='Buy Signal'
))

fig.add_trace(go.Scatter(
    x=[signal[0] for signal in sells],
    y=[signal[1] for signal in sells],
    mode='markers',
    marker=dict(size=10, color='red', symbol='triangle-down'),
    name='Sell Signal'
))

fig.update_layout(
    title="Buy/Sell Signals",
    xaxis_title="Date",
    yaxis_title="Price",
    template="plotly_white",
    height=600,
    width=1000
)

fig.show()

In [30]:
# Create a Plotly figure
fig = go.Figure()

for strat_name, data_dict in strategy_dict.items():
    run_results = data_dict['results']
    
    # Usually returns is a list of strategy instances; we assume one strategy instance
    strategy_instance = run_results[0]
    
    # Extract the TimeReturn analyzer
    time_return_dict = strategy_instance.analyzers.timereturn.get_analysis()
    
    # Convert to a pandas Series
    returns_series = pd.Series(time_return_dict)
    
    # Calculate cumulative returns: (1 + r).cumprod() - 1
    cumulative_returns = (1 + returns_series).cumprod() - 1
    
    # Plot with Plotly
    fig.add_trace(go.Scatter(
        x=cumulative_returns.index,
        y=cumulative_returns.values,
        mode='lines',
        name=f"{strat_name} (Cumulative)"
    ))

# Update figure layout
fig.update_layout(
    title="Strategy Comparison: Cumulative Returns",
    xaxis_title="Date",
    yaxis_title="Cumulative Return",
    template="plotly_white",
    width=900,
    height=500
)

# Show the figure
fig.show()


In [ ]:
class HybridMeanReversionVWMACDStrategy(bt.Strategy):
    params = (
        # Mean Reversion Parametreleri
        ('lookback', 50),  # Hareketli ortalama için periyot
        ('threshold', 1.5),  # Z-score eşiği

        # VWMACD Parametreleri
        ('short_period', 5),
        ('long_period', 10),
        ('signal_period', 9),
    )

    def _init_(self):
        # Mean Reversion için Hareketli Ortalama ve Standart Sapma
        self.mean = bt.indicators.SimpleMovingAverage(self.data.close, period=self.params.lookback)
        self.std = bt.indicators.StandardDeviation(self.data.close, period=self.params.lookback)

        # VWMA hesaplama
        self.vwma_short = bt.indicators.WeightedMovingAverage(self.data.close, period=self.params.short_period)
        self.vwma_long = bt.indicators.WeightedMovingAverage(self.data.close, period=self.params.long_period)

        # VMACD ve sinyal hattı
        self.vmacd = self.vwma_short - self.vwma_long
        self.signal = bt.indicators.EMA(self.vmacd, period=self.params.signal_period)

    def next(self):
        # Z-score hesaplama
        zscore = (self.data.close[0] - self.mean[0]) / self.std[0]

        # Alış sinyali: Mean Reversion alt seviyede ve VMACD yukarı momentum gösteriyor
        if zscore < -self.params.threshold and self.vmacd[0] > self.signal[0] and not self.position:
            self.buy()

        # Satış sinyali: Mean Reversion üst seviyede ve VMACD aşağı momentum gösteriyor
        elif zscore > self.params.threshold and self.vmacd[0] < self.signal[0] and self.position:
            self.sell()